In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

print("Package loaded!")

In [ ]:
_data_dir = "./mnist/"

mnist = input_data.read_data_sets(_data_dir, one_hot=True)

img_test = mnist.test.images
label_test = mnist.test.labels

In [ ]:
NUM_CLASSES = 10
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE    #784
NUM_HIDDEN1_NODE = 256
NUM_HIDDEN2_NODE = 128

stddev = 0.1

x = tf.placeholder("float", [None, IMAGE_PIXELS])
y = tf.placeholder("float", [None, NUM_CLASSES])
W1 = tf.Variable(tf.random_normal([IMAGE_PIXELS, NUM_HIDDEN1_NODE], stddev=stddev))
b1 = tf.Variable(tf.zeros([NUM_HIDDEN1_NODE]))
W2_ = tf.Variable(tf.random_normal([NUM_HIDDEN1_NODE, NUM_CLASSES], stddev=stddev))
b2_ = tf.Variable(tf.zeros([NUM_CLASSES]))
W2 = tf.Variable(tf.random_normal([NUM_HIDDEN1_NODE, NUM_HIDDEN2_NODE], stddev=stddev))
b2 = tf.Variable(tf.zeros([NUM_HIDDEN2_NODE]))
W3 = tf.Variable(tf.random_normal([NUM_HIDDEN2_NODE, NUM_CLASSES], stddev=stddev))
b3 = tf.Variable(tf.zeros([NUM_CLASSES]))

def inference_mlp1(data_in, weight_1, bias_1, weight_2, bias_2):
    hypothesis_1 = tf.matmul(data_in, weight_1) + bias_1
    gz_1 = tf.sigmoid(hypothesis_1)
    hypothesis_2 = tf.matmul(gz_1, weight_2) + bias_2
    return hypothesis_2

def inference_mlp2(data_in, weight_1, bias_1, weight_2, bias_2, weight_3, bias_3):
    hypothesis_1 = tf.matmul(data_in, weight_1) + bias_1
    gz_1 = tf.sigmoid(hypothesis_1)
    hypothesis_2 = tf.matmul(gz_1, weight_2) + bias_2
    gz_2 = tf.sigmoid(hypothesis_2)
    hypothesis_3 = tf.matmul(gz_2, weight_3) + bias_3
    return hypothesis_3

def cost(logits, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))

def training(loss, learning_rate):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train = optimizer.minimize(loss, global_step=global_step)
    return train

def evaluation(logits, labels):
    pred = tf.equal(tf.argmax(tf.nn.softmax(logits), 1), tf.argmax(labels, 1))
    accr = tf.reduce_mean(tf.cast(pred, "float"))
    return accr

In [ ]:
learning_rate = 0.001

#logits = inference_mlp1(x, W1, b1, W2_, b2_)
logits = inference_mlp2(x, W1, b1, W2, b2, W3, b3)
loss = cost(logits, y)
train = training(loss, learning_rate=learning_rate)
eval_correct = evaluation(logits, y)

init = tf.initialize_all_variables()

In [ ]:
n_samples = mnist.train.num_examples
batch_size = 200
total_batch = int(n_samples / batch_size)
start = time.time()
total_epoch = 30

# SESSION
with tf.Session() as sess:
    sess.run(init)
    # MINI-BATCH LEARNING
    for epoch in range(total_epoch):
        avg_cost = 0.
        for i in range(total_batch + 1):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed = {x:batch_xs, y:batch_ys}
            sess.run(train, feed)
            avg_cost += sess.run(loss, feed) / batch_size
            
        # DISPLAY
        if epoch % 5 == 0:
            feeds_train = {x:batch_xs, y:batch_ys}
            feeds_test = {x: img_test, y: label_test}
            train_acc = sess.run(eval_correct, feeds_train)
            test_acc = sess.run(eval_correct, feeds_test)
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
                   % (epoch, total_epoch, avg_cost, train_acc, test_acc))
print ("DONE")
print time.time() - start